In [36]:
import os

from huggingface_hub import login
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
# authentication with huggingface
# create token at https://huggingface.co/settings/tokens (create as read)
# token should be stored locally, so technically login is only needed one time
login()

In [20]:
# download specific dataset files; entire archive is only ~250 mb
articles = load_dataset("Cofacts/line-msg-fact-check-tw", "articles") # article contents and some meta info
article_replies = load_dataset("Cofacts/line-msg-fact-check-tw", "article_replies") # 'join table' for articles and replies with added meta info

Generating train split: 0 examples [00:00, ? examples/s]

In [21]:
articles_replies_df = pd.merge(articles["train"].to_pandas(), article_replies["train"].to_pandas(), left_on="id", right_on="articleId", how="left")
articles_replies_df.head()
# replyType is the label assigned to the article by the human editor

,id,articleType,status_x,text,normalArticleReplyCount,createdAt_x,updatedAt_x,lastRequestedAt,userIdsha256_x,appId_x,references,articleId,replyId,userIdsha256_y,negativeFeedbackCount,positiveFeedbackCount,replyType,appId_y,status_y,createdAt_y,updatedAt_y
0,14h550ymn3m3u,TEXT,NORMAL,BA.5病毒引發的不只「縮短潛伏期，增加R0值」，還有易使人「再感染（Reinfecion）...,1.0,2022-09-05T05:26:33.360Z,2022-09-05T05:26:33.360Z,2022-09-05T05:26:33.381Z,3753ca22ca96f5fc1f13e62291f28e405d56b1a976b21d...,RUMORS_LINE_BOT,LINE,14h550ymn3m3u,6oo8DYMBv5it-Cx_8405,6892d0026181e95d034bf8781025afbf395d57cc88ac05...,0.0,3.0,RUMOR,WEBSITE,NORMAL,2022-09-05T10:40:41.789Z,2022-09-05T10:40:41.789Z
1,326xjpkjbf01i,TEXT,NORMAL,https://youtu.be/xK9NzL3PkdE,0.0,2022-09-08T12:34:27.602Z,2022-09-08T12:34:27.602Z,2022-09-08T12:34:27.634Z,243b5897c14f02fb5b92a9e4f4cc39d5fb84ff16173add...,RUMORS_LINE_BOT,LINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,oupi0eu9aocr,TEXT,NORMAL,被取消的航班\n\n2018年的一天，我從洛杉磯回國，乘坐凌晨一點的國航航班，馬上要登機了，...,1.0,2021-11-13T16:36:56.548Z,2021-11-13T16:36:56.548Z,2022-04-27T08:52:17.115Z,4a266a6fdefc88e59eef644402e97c7f4c8196becca686...,RUMORS_LINE_BOT,LINE,oupi0eu9aocr,MYotCYMBv5it-Cx_gIpH,fc9652aa205dab75e19ecff420945e95aa3909a54b0d9a...,0.0,1.0,OPINIONATED,WEBSITE,NORMAL,2022-09-04T15:45:20.464Z,2022-09-04T15:45:20.464Z
3,my5ep5z69tql,TEXT,NORMAL,你好👱🏻‍♀️\n\n簡單跟你說一下公司工作內容：\n我們是做購物系統訂單的工作，主要就是增...,1.0,2022-08-11T20:53:15.644Z,2022-08-11T20:53:15.644Z,2022-09-07T11:02:00.344Z,8a8a319bea0b3bff580d8507ae6470ba37e87791171886...,RUMORS_LINE_BOT,LINE,my5ep5z69tql,y22DlmsBFV14knB4ErTJ,d16417c7ce4ab67ac5a7901ce62621445db3e06da5da6f...,0.0,1.0,RUMOR,WEBSITE,NORMAL,2022-09-07T22:59:48.921Z,2022-09-07T22:59:48.921Z
4,3dnh713ikpf3c,TEXT,NORMAL,先和你簡單介紹\n我們誠徵線上打工小幫手\n工作內容有：聽寫打字、頁面排版、商品評論刷流量、...,1.0,2022-09-07T10:10:41.742Z,2022-09-07T10:10:41.742Z,2022-09-07T10:10:41.761Z,47e90b2538a5615b323f50571b77d91f7f105e55684aa1...,RUMORS_LINE_BOT,LINE,3dnh713ikpf3c,H8Hz4WsBqwaEkHKwtyFm,d16417c7ce4ab67ac5a7901ce62621445db3e06da5da6f...,0.0,1.0,RUMOR,WEBSITE,NORMAL,2022-09-07T23:00:19.207Z,2022-09-07T23:00:19.207Z


In [35]:
articles_replies_df["replyType"].value_counts()

replyType
RUMOR          47408
NOT_RUMOR      16613
NOT_ARTICLE    15430
OPINIONATED    10955
Name: count, dtype: int64